<a href="https://colab.research.google.com/github/uoacapstonegroup6/CapstoneUOATeam6/blob/main/AdrenalMNIST3D_with_basic_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install if required
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=46015adc8f6a6dbf327265d135c394dbe9950d817be55ee1e10001a58c009f1a
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [2]:
#Importing necessary packages

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
import medmnist
from medmnist import INFO, Evaluator

In [4]:
#Creating dataflags

data_flag = 'adrenalmnist3d'
download = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

# load the data
train_dataset = DataClass(split='train', download=download, size=64)
test_dataset = DataClass(split='test', download=download, size=64)
val_dataset = DataClass(split='val', download=download, size=64)

100%|██████████| 1880591/1880591 [00:00<00:00, 1896071.75it/s]


Using downloaded and verified file: /root/.medmnist/adrenalmnist3d_64.npz
Using downloaded and verified file: /root/.medmnist/adrenalmnist3d_64.npz


In [5]:
#Creating a function which converts a 3D image into 2D image in tensor format
def image_3d_to_2d(dataset):

  #Initializing empty lists to store 2D images
  images = []
  labels = []
  index = []
  print("Initial 3D images:",len(dataset))

  for i in range(len(dataset)):           #This loop runs for a few iterations as many as no. of 3D images
    x, y = train_dataset[i]               #Extracting the image tensor (x) and label (y) from one 3D image
    for j in range(len(x[0])):            #Getting into first batch of the selected 3D image. There will be only 1 batch and hence x[0] and len(x[0]) gives number of slices
      img = x[0][j]                       #Array for every slice (j) is captured
      arr = (img).astype(np.float32)      #Converting the normalized array into float
      tensor = torch.tensor(arr)          #Converting array into tensor
      images.append(tensor)               #Appending the image tensor into a list
      labels.append(y[0])                 #Appending the label for the image into a list
      index.append(i)                     #Appending the 3D image index into a list
  print("Sliced 2D images:",len(images))
  return images, labels, index

In [6]:
#Running the above function for train, test and validation.
#Since the size=28, every 3D image has 28 slices

train_2d = image_3d_to_2d(train_dataset)
test_2d = image_3d_to_2d(test_dataset)
val_2d = image_3d_to_2d(val_dataset)

Initial 3D images: 1188
Sliced 2D images: 76032
Initial 3D images: 298
Sliced 2D images: 19072
Initial 3D images: 98
Sliced 2D images: 6272


In [7]:
#Once sliced up, images are saved as .pth file

images, labels, index = train_2d
torch.save({'images': images, 'labels': labels, 'index':index}, 'fracture_train_2d.pth')
images, labels, index = test_2d
torch.save({'images': images, 'labels': labels, 'index':index}, 'fracture_test_2d.pth')
images, labels, index = val_2d
torch.save({'images': images, 'labels': labels, 'index':index}, 'fracture_val_2d.pth')

In [8]:
# Load the saved dataset
data_train = torch.load('fracture_train_2d.pth')
data_test = torch.load('fracture_test_2d.pth')
data_val = torch.load('fracture_val_2d.pth')

# Convert the loaded images and labels to tensors
tr_images = torch.stack(data_train['images'])  # Stack list of tensors into a single tensor
tr_images = tr_images.unsqueeze(1)              #Forcefully adding n_channel=1
tr_images_rgb = tr_images.repeat(1, 3, 1, 1)
tr_labels = torch.tensor(data_train['labels'])  # Convert list of labels into a tensor
# if tr_labels.ndimension() == 1:  # If it's (num_samples,)
    # num_classes = 3  # Replace with actual number of classes
    # tr_labels = torch.nn.functional.one_hot(tr_labels, num_classes=num_classes).float()
tr_index = torch.tensor(data_train['index'])  # Convert list of indices into a tensor

ts_images = torch.stack(data_test['images'])  # Stack list of tensors into a single tensor
ts_images = ts_images.unsqueeze(1)              #Forcefully adding n_channel=1
ts_images_rgb = ts_images.repeat(1, 3, 1, 1)
ts_labels = torch.tensor(data_test['labels'])  # Convert list of labels into a tensor
# if ts_labels.ndimension() == 1:  # If it's (num_samples,)
#     num_classes = 3  # Replace with actual number of classes
#     ts_labels = torch.nn.functional.one_hot(ts_labels, num_classes=num_classes).float()
ts_index = torch.tensor(data_test['index'])  # Convert list of indices into a tensor

val_images = torch.stack(data_val['images'])  # Stack list of tensors into a single tensor
val_images = val_images.unsqueeze(1)              #Forcefully adding n_channel=1
val_images_rgb = val_images.repeat(1, 3, 1, 1)
val_labels = torch.tensor(data_val['labels'])  # Convert list of labels into a tensor
val_index = torch.tensor(data_val['index'])  # Convert list of indices into a tensor

# Create a TensorDataset from the loaded data
dataset_train = data.TensorDataset(tr_images_rgb, tr_labels, tr_index)
dataset_test = data.TensorDataset(ts_images_rgb, ts_labels, ts_index)
dataset_val = data.TensorDataset(val_images_rgb, val_labels, val_index)

# Create a DataLoader for batching and shuffling
train_loader = data.DataLoader(dataset_train, batch_size=128, shuffle=True)
test_loader = data.DataLoader(dataset_test, batch_size=128, shuffle=True)
val_loader = data.DataLoader(dataset_val, batch_size=128, shuffle=True)



<ipython-input-8-52d39c6dc2ed>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_train = torch.load('fracture_train_2d.pth')
<ipython-input-8-52d39c6dc2ed>:3: FutureWarn

In [ ]:
# To see each batch of images
# for batch in test_loader:
#     batch_images, batch_labels, batch_index = batch
#     print(batch_images.shape)  # Should be (batch_size, 1, 28, 28)
#     print(batch_labels)

In [9]:
!nvidia-smi

Wed Sep 18 06:12:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
!git clone https://github.com/Omid-Nejati/MedViT.git
%cd /content/MedViT

Cloning into 'MedViT'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 176 (delta 87), reused 134 (delta 61), pack-reused 1 (from 1)
Receiving objects: 100% (176/176), 820.48 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/MedViT


In [11]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 13.4 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61395 sha256=0c1834770b95a1f3ac1fa3a2

In [12]:
from MedViT import MedViT_small as tiny, MedViT_base, MedViT_large

model = tiny()

initialize_weights...


In [13]:
# Assuming you're using a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
model = model.to(device)

In [14]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=3, bias=True)

In [15]:
model = model.to(device)

In [16]:
# evaluation

# task = 'multi-label, binary-class'
model.eval()
y_true = torch.tensor([]).cuda()
y_score = torch.tensor([]).cuda()

data_loader = test_loader

with torch.no_grad():
    for inputs, targets, index in data_loader:
        inputs, targets, index = inputs.cuda(), targets.cuda(), index.cuda()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            outputs = outputs.softmax(dim=-1)
            # print(1)
        else:
            targets = targets.squeeze().long()
            outputs = outputs.softmax(dim=-1)
            targets = targets.float().resize_(len(targets), 1)
            # print(0)

        y_true = torch.cat((y_true, targets), 0)
        y_score = torch.cat((y_score, outputs), 0)

    y_true = y_true.cpu().numpy()
    y_score = y_score.detach().cpu().numpy()

In [17]:
actuals = y_true.squeeze().tolist()
predictions = []
y_score = y_score.tolist()
for i in range(len(y_score)):
  predictions.append(y_score[i].index(max(y_score[i])))

In [18]:
correct_predictions = sum(a==p for a, p in zip(actuals, predictions))
accuracy = (correct_predictions / len(actuals)) * 100

In [19]:
accuracy

23.825503355704697

In [20]:
precision = precision_score(actuals, predictions, average='macro')
recall = recall_score(actuals, predictions, average='macro')
f1 = f1_score(actuals, predictions, average='macro')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 23.825503355704697
Precision: 0.11912751677852348
Recall: 0.5
F1 Score: 0.19241192411924118
